In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import statistics
plt.rc('figure', facecolor='w', figsize=[4,3])

In [ ]:
kmers = list(map(int,"11 13 15 17 19 21 25 31 41 51 61 81 101 151 201 251 301 501".split()))

data_kmers = []
for k in kmers:
    abundances = {}
    mean = 0
    nb_items = 0
    genome_abundance_1_total = 0
    for x,line in enumerate(open("chm13v2.0-"+str(k)+".histo2D")):
        l = list(map(int,line.split()[1:]))
        assert(int(line.split()[0].strip(":")) == x)
        abundances[x]=l
        genome_abundance_1_total += l[1]
        if x == 0:
            genome_abundance_1_missing = l[1]
    data_kmers += [(k,genome_abundance_1_missing/genome_abundance_1_total)]

In [ ]:

df = pd.DataFrame([x for x in data_kmers],columns=("k","nb_kmers_abundance_1_in_genome"))

print(df)
sns_plot = sns.lineplot(x=df["k"], y=df["nb_kmers_abundance_1_in_genome"])

#sns_plot.set(xscale="log")
#sns_plot.set(xticks=[x/100 for x in range(0,10+1,2)])
#sns_plot.set_xticklabels([str(x-2) for x in range(0,10+3,2)])

ylabels = ['{:}%'.format(int(x*100))  for x in sns_plot.get_yticks()]
sns_plot.set_yticklabels(ylabels)

#plt.xticks(sns_plot.get_xticks(), sns_plot.get_xticks() * 100)
sns_plot.set(xlabel='k value', ylabel='')
#sns_plot.get_legend().remove()
plt.title('Uniquely occurring reference k-mers \n missing from HiFi DeepConsensus reads')

fig = sns_plot.get_figure()
fig.savefig("seeds_kmers.pdf", bbox_inches='tight', dpi =300)


In [ ]:
ksizes = list(map(int,"2 4 6 8 10".split()))

data_kminmers = []
for k in ksizes:
    abundances = {}
    mean = 0
    nb_items = 0
    genome_abundance_1_total = 0
    for x,line in enumerate(open(f"2DHisto-k{k}-d0.01-l31.hist2D")):
        l = list(map(int,line.split()))
        abundances[x]=l
        genome_abundance_1_total += l[1]
        if x == 0:
            genome_abundance_1_missing = l[1]
    data_kminmers += [(k,genome_abundance_1_missing/genome_abundance_1_total)]

In [ ]:
df = pd.DataFrame([x for x in data_kminmers],columns=("k","nb_kminmers_abundance_1_in_genome"))

print(df)
sns_plot = sns.lineplot(x=df["k"], y=df["nb_kminmers_abundance_1_in_genome"])

#sns_plot.set(xscale="log")
#sns_plot.set(xticks=[x/100 for x in range(0,10+1,2)])
#sns_plot.set_xticklabels([str(x-2) for x in range(0,10+3,2)])

ylabels = ['{:}%'.format(int(x*100))  for x in sns_plot.get_yticks()]
sns_plot.set_yticklabels(ylabels)

#plt.xticks(sns_plot.get_xticks(), sns_plot.get_xticks() * 100)
sns_plot.set(xlabel='k value', ylabel='')
#sns_plot.get_legend().remove()
plt.title('Uniquely occurring reference k-min-mers \n missing from HiFi DeepConsensus reads')


fig = sns_plot.get_figure()

# create second Axes. Note the 0.0 height
ax2 = fig.add_axes((0.15,-0.1,0.72,0.0))
ax2.yaxis.set_visible(False) # hide the yaxis
new_tick_locations = np.array([0,.25, .5,.75,1])
def tick_function(X):
    ksizes = list(map(int," 2 4 6 8 10".split()))
    density=0.01
    return ["%d" % (ksizes[i]/density) for i,z in enumerate(X)]
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
ax2.set_xlabel("Equivalent k span in bases")
plt.show()

fig.savefig("seeds_kminers.pdf", bbox_inches='tight', dpi =300)


In [ ]:
#attempt to merge both plots

def k_to_kspan(k):
    density=0.01
    return k/density

def k_to_kweight(k):
    l=31
    return k*l

data = {}
for x,y in data_kmers:
    data[x]=(x,y,"k-mers")
for x,y in data_kminmers:
    kspan = k_to_kspan(x)
    kweight = k_to_kweight(x)
    data[kspan] = (kweight,y,"k-min-mers")

ktype = "kspan"
ktype = "kweight"

print(data)
df = pd.DataFrame([(x,data[x][0],data[x][1],data[x][2]) for x in data],columns=("kspan","kweight","perc_kmers_abundance_1_in_genome","type"))
  
sns_plot = sns.lineplot(df,x=ktype,y="perc_kmers_abundance_1_in_genome",hue="type")

#sns_plot.set(xscale="log")
#sns_plot.set(xticks=[x/100 for x in range(0,10+1,2)])
#sns_plot.set_xticklabels([str(x-2) for x in range(0,10+3,2)])

ylabels = ['{:}%'.format(int(x*100))  for x in sns_plot.get_yticks()]
sns_plot.set_yticklabels(ylabels)

#plt.xticks(sns_plot.get_xticks(), sns_plot.get_xticks() * 100)
if ktype == "kspan":
    sns_plot.set(xlabel='Equivalent k-mer length', ylabel='')
else:
    sns_plot.set(xlabel='Seed weight', ylabel='')
#sns_plot.get_legend().remove()
plt.title('Percentage of seeds that occur uniquely \n in the genome yet are absent from reads')

fig = sns_plot.get_figure()
if ktype == "kspan":
    # create second Axes. Note the 0.0 height
    ax2 = fig.add_axes((0.3,-0.1,0.56,0.0)) # i didn't determine those params automatically, these are just empirically set
    ax2.yaxis.set_visible(False) # hide the yaxis
    def tick_function(X):
        ksizes = list(map(int,"2 4 6 8 10".split()))
        return ["%d" % (ksizes[i]) for i,z in enumerate(X)]
    new_tick_locations = np.array([0,.25, .5,.75,1])
    ax2.set_xticks(new_tick_locations)
    ax2.set_xticklabels(tick_function(new_tick_locations))
    ax2.set_xlabel("k-min-mer's k value (density=0.01)")
plt.show()

fig.savefig("seeds_2D.pdf", bbox_inches='tight', dpi =300)
